#Carga de información GITHUB

In [ ]:
try:
    !rm -rf AprendizMaquina
except:
    print("No existe la ruta aún")



!git clone 'https://github.com/pikey-msc/AprendizMaquina'


Cloning into 'AprendizMaquina'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 182 (delta 23), reused 0 (delta 0), pack-reused 66
Receiving objects: 100% (182/182), 10.68 MiB | 19.46 MiB/s, done.
Resolving deltas: 100% (40/40), done.


#Carga de librerías

In [ ]:
#Cargar módulo de R en Python
# A bit of imports
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load in the r magic
%reload_ext rpy2.ipython
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'

In [ ]:
%%R
#install.packages("dplyr")
library(dplyr) # "Libreria utilizada para manejar datos"
library(rpart)
install.packages("rpart.plot")
library(rpart.plot)
install.packages("caret")
library(caret)
install.packages("e1071")
library(e1071)
install.packages("ModelMetrics")
library(ModelMetrics)
install.packages("party")
library(party)
# install.packages("boot")
# library(boot)
# install.packages("ipred")
# library(ipred)
# install.packages("randomForest")
# library(randomForest)
# library(ggplot2)
# install.packages("ISLR")
# library(ISLR)



Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union


(as ‘lib’ is unspecified)







	‘/tmp/Rtmpzt0Yuw/downloaded_packages’

(as ‘lib’ is unspecified)

































































































































	‘/tmp/Rtmpzt0Yuw/downloaded_packages’



(as ‘lib’ is unspecified)







	‘/tmp/Rtmpzt0Yuw/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/Rtmpzt0Yuw/downloaded_packages’

Attaching package: ‘ModelMetrics’



    confusionMatrix, precision, recall, sensitivity, specificity



    kappa


(as ‘lib’ is unspecified)



























































	‘/tmp/Rtmpzt0Yuw/downloaded_packages’







Attaching package: ‘zoo’



    as.Date, as.Date.numeric





In [ ]:
%%R
#--------------Algoritmo kNN   Vecinos más cercanos----------------------

#  El conjunto de datos contiene 50 muestras de cada una de tres especies de Iris (Iris setosa, Iris virginica e Iris versicolor).
#  Se midió cuatro rasgos de cada muestra: lo largo y lo ancho del sépalos y pétalos, en centímetros.
#  Basado en la combinación de estos cuatro rasgos, Fisher desarrolló un modelo discriminante lineal para distinguir entre una especie y otra
#  Pasos:
#    
#    Paso 1: Obtener una muestra de 6 elemenos con datos setosa y versicolor
#  
#  Paso 2: Utilizar un punto k=1 definido en X
#  
#  Paso 3: Calcular las distacias euclideanas entre los puntos y k
#  
#  Paso 4: Clasificar la información de acuerdo a las distancias de mayor a menor y dividir la información en k=1  
#  ALGORITMO kNN
#  Escoger el número de vecinos cercanos, es decir, determinar el valor de k, representa un valor muy significativo para determinar la eficiencia del modelo. Por tanto, la selección de k determinará que tan bien los datos pueden ser utilizados para generalizar los algoritmos del modelo KNN
#  Si k es muy pequeño el modelo será muy sentitivo a puntos que son atípicos o que son ruido (datos corruptos).
#  Si k es muy grande, el modelo tiende a asignar siempre a la clase más grande.
#  Sin embargo, en R es posible obtener el valor de k óptimo
#  ALGORITMO kNN
#  La base "Iris" puededescargarse mediante R.   

Iris  <-iris  #  Guardamos  los  datos
#  summary(Iris)
#  Cambiamos  el  nombre  de  las  columnas  por:

colnames(Iris)  <- c("Largo_Sepalo","Ancho_Sepalo","Largo_Petalo","Ancho_Petalo",  "Especies")

#  Visualización de Iris
plot(Iris$Largo_Petalo,  Iris$Ancho_Petalo,  pch=21,bg=c("red","green3","blue")[unclass(Iris$Especies)],  main="Iris",xlab  =  "Largo  Petalo",ylab  ="Ancho  Petalo")

#ALGORITMO kNN
# Iris: Prueba / Test

# Dividir los datos en:
#   ** Datos de prueba: Datos historicos para ajustar el modelo

#  ** Datos de test: Datos históricos para evaluar (probar) el modelo

#  Barajeamos la información de iris

#  Fijamos  la  semilla  aleatoria  de  los  datos

set.seed(42)

#  Barajeamos  de  manera  aleatoria  los  indices  de  los  renglones

renglones  <-sample(nrow(Iris))

#  De  forma  aleatoria  reordenamos  los  datos

Iris  <-Iris[renglones,  ]


#División datos: prueba/test

#Opción 1: División por porcentaje 80% - 20%
#  Paso  1.  Fijamos  la  variable  de  división  de  80%  en  los  datos

division  <- round(nrow(Iris)  *  .80)
#Paso  2.  Tomamos  el  80%  de  los  datos  como:  Datos  prueba

prueba  <- Iris[1:division,]

nrow(prueba)

#Paso  3.  Tomamos  el  20%  de  los  datos  como:  Datos  test
test  <-Iris[(division  +  1):nrow(Iris),  ]

nrow(test)



#División datos: prueba/test

#Opción 2: División por número de elementos 120 - 30
#  Paso  1.  Fijamos  las  variables  de  división  de  120  datos

division  <-sample(1:150,120)
#Paso  2.  Tomamos  120  datos  como:  Datos  prueba

prueba  <-Iris[division,]

nrow(prueba)

#Paso  3.  Tomamos  30    datos  como:  Datos  test

test  <-Iris[-division,]

nrow(test)

#Calculo del modelo kNN en R
# install.packages("kknn")
library(kknn)          #  Para  poder  utilizar  el  Modelo  kNN  ponderado
library(class)        #  Opcional  para  poder  utilizar  kNN

#  Ajuste  del  modelo  "kNN""

modelo=train.kknn(prueba$Especies~.,data=prueba,K=8)
modelo$best.parameters
#  Para  este  ejercicio  utilizaremos  K  =  8

#  Predicción  de  los  datos

p  <-predict(modelo,test[,-5])

p  


#Calcular la Matriz de Confusión

# Crea  la  matriz  de  confusión 

#Crea  la  matriz  de  confusión
confusionMatrix(p,test[["Especies"]])
table(p,  test[["Especies"]])


#Calcular la "k"" óptima

#  Paso  1:  Definimos  control

ctrl  <-trainControl(method="repeatedcv",repeats  =  3)
#  Paso  2:  Calculamos  la  k  óptima
knnFit  <- train(Especies  ~  .,
                 data  =  prueba,
                 method  =  "knn",
                 trControl  =  ctrl,
                 tuneLength  =  20)
knnFit



#Gráfica la "k" óptima

plot(knnFit)

In [ ]:
%%R
#Ejercicio 113: Billetes Banco

#Los datos son extraídos de imagenes que fueron tomadas de billetes de banco genuinos y falsificados.

#  Para la digitalización, se utilizó una camara industrial usualmente utilizada para inspección de impresión.

# Las imagenes tienen 400 X 400 pixeles con una resolución de 660 dpi.

#  Se utilizo una transformación de imagenes para almacenar la información en una base de datos.

# ALGORITMO kNN
#  Resumen

billete <- read.table("data_banknote_authentication.txt", header  =  F,  sep =  ",")

summary(billete)


#  Fijamos  la  semilla  aleatoria  de  los  datos

set.seed(42)

#  Barajeamos  de  manera  aleatoria  los  indices  de  los  renglones

renglones  <-sample(nrow(billete))

#  De  forma  aleatoria  reordenamos  los  datos 

billete  <-billete[renglones,  ]


#  División por número de elementos 1000 -372

#  Paso  1.  Fijamos  las  variables  de  división  de  1000  datos 

division  <-sample(1:1372,1000)
#Paso  2.  Tomamos  1000  datos  como:  Datos  prueba

prueba  <-billete[division,]

nrow(prueba)

#Paso  3.  Tomamos  372  de  los  datos  como:  Datos  test

test  <-billete[-division,]

nrow(test)  


#  Calculo del modelo kNN en R

#  Ajuste  del  modelo  "kNN""

modelo  <-knn(prueba[,1:4],test[,1:4],prueba[,5],k=3,prob=TRUE)
#Gráfica  del  modelo
summary(modelo)

plot(modelo)  


#  Predicción  de  los  datos
#Crea  la  matriz  de  confusión  

confusionMatrix(table(test[,5],  modelo))


#Calcular la "k"" óptima
#  Paso  1:  Definimos  control

ctrl  <-trainControl(method="repeatedcv",repeats  =  3)

#  Paso  2:  Calculamos  la  k  óptima

knnFit  <-train(as.factor(V5)  ~  .,  data  =  prueba,  method  =  "knn",    trControl  =  ctrl,  tuneLength  =  20)  

knnFit

plot(knnFit)


p  <-predict(knnFit,test)

p


confusionMatrix(as.factor(p),as.factor(test$V5))


